In [36]:
from __future__ import print_function, division
import numpy as np
np.random.seed(42)
import tensorflow as tf
import pdb
import fxns

In [37]:
# data locations
raw_data_path = '../data/shuffled_examples'
outbase = '../output/shuffled_examples'
train_out_path = '%s/texttrain' % outbase
valid_out_path = '%s/textvalidate' % outbase
test_out_path = '%s/texttest' % outbase
motif_path = '%s/../motifs' % raw_data_path

# training hyperparameters
batch_size = 50
num_batches_in_train = int(434786 / batch_size)
num_epochs = 2
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

# retrieving models
global_step_to_load = None

In [59]:
tf.reset_default_graph()

# get training data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('inputs are of size', info['seq_len'])

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('outputs are of size', info['label_len'])

reload(fxns)
print(test_out_path)
(seqt, labelt), info = fxns.get_seq_and_label(test_out_path, num_epochs=None)
seqt_batch, labelt_batch = tf.train.shuffle_batch([seqt, labelt],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

inputs are of size 4000
outputs are of size 24
../output/shuffled_examples/texttest


In [60]:
# # get model -- logistic
# import logreg_model
# modelname='logreg'
# X, Y, loss, logits = logreg_model.get_logreg_model(info['seq_len'], info['label_len'])

# get model -- generic convnet
import motif_model; reload(motif_model)
filters = motif_model.get_motifs(motif_path)
modelname='motifconv'
#conv_infos = [(7,(1,20),(2,2),4),(5,(1,20),(2,2),7)]#, (8,(1,20),(2,2),5)]
conv_infos = [(14,(1,21),(2,2),2,4),(14,(1,8),(2,2),2,14),(7,(1,5),(2,2),2,14)]#, (8,(1,20),(2,2),5)]
X, Y, loss, logits = motif_model.get_motif_model(info['seq_len'], info['label_len'], conv_infos, filters)

('max filter length set to', 21)
500.0
250.0
125.0
4000 h 15 125.0


In [61]:
# define optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,
                                                 name='SGD-Optimizer')
    update = optimizer.minimize(loss)

# define other summaries we want
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

In [49]:
if tf.gfile.Exists('log/' + modelname):
   tf.gfile.DeleteRecursively('log/' + modelname) 

# train

print_every = 25
save_every = 250

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    
    if global_step_to_load is None:
        global_step = 0
    else:
        saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load))
        global_step = global_step_to_load
    
#    print(layer)
    
    writer = tf.summary.FileWriter('log/' + modelname + '/train', sess.graph)
    writerv = tf.summary.FileWriter('log/' + modelname + '/valid')
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    if global_step_to_load is None:
        sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(num_batches_in_train):
            batch_X, batch_Y = sess.run([seq_batch, label_batch])
            batch_Xv, batch_Yv = sess.run([seqv_batch, labelv_batch])
            _, _loss = sess.run([update, loss],
                            feed_dict={X: batch_X, Y: batch_Y})
            total_loss += _loss #/ num_batches_in_train
            
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            summaryv = sess.run(summary_op, feed_dict={X: batch_Xv,
                                                    Y: batch_Yv})
#            writer.add_summary(summary, global_step=epoch*num_batches_in_train + i)
#            writerv.add_summary(summaryv, global_step=epoch*num_batches_in_train + i)
            writer.add_summary(summary, global_step=global_step)
            writerv.add_summary(summaryv, global_step=global_step)
            
            global_step += 1
            
            if global_step % save_every == 0:
                print('saving')
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
                saver.save(sess, 'log/%s' % modelname, global_step=global_step)
                
            if global_step % print_every == 0:
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
        
        print('saving')
        print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))

        saver.save(sess, 'log/%s' % modelname, global_step=global_step)
    
    writer.close()
    coord.request_stop()
    coord.join(threads)

Epoch: 0 global_step 25 i 24 Avg loss in epoch(incomplete): 0.76439155817
Epoch: 0 global_step 50 i 49 Avg loss in epoch(incomplete): 0.744075440168
Epoch: 0 global_step 75 i 74 Avg loss in epoch(incomplete): 0.726037234465
Epoch: 0 global_step 100 i 99 Avg loss in epoch(incomplete): 0.709554651976
Epoch: 0 global_step 125 i 124 Avg loss in epoch(incomplete): 0.693957925797
Epoch: 0 global_step 150 i 149 Avg loss in epoch(incomplete): 0.678978453875
Epoch: 0 global_step 175 i 174 Avg loss in epoch(incomplete): 0.664363247326
Epoch: 0 global_step 200 i 199 Avg loss in epoch(incomplete): 0.649966078401
Epoch: 0 global_step 225 i 224 Avg loss in epoch(incomplete): 0.635458083683
saving
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.621021033049
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.621021033049
Epoch: 0 global_step 275 i 274 Avg loss in epoch(incomplete): 0.606645810279
Epoch: 0 global_step 300 i 299 Avg loss in epoch(incomplete): 0.59226319174
E

Epoch: 0 global_step 2425 i 2424 Avg loss in epoch(incomplete): 0.238103302066
Epoch: 0 global_step 2450 i 2449 Avg loss in epoch(incomplete): 0.2373600893
Epoch: 0 global_step 2475 i 2474 Avg loss in epoch(incomplete): 0.236629561686
saving
Epoch: 0 global_step 2500 i 2499 Avg loss in epoch(incomplete): 0.235927727032
Epoch: 0 global_step 2500 i 2499 Avg loss in epoch(incomplete): 0.235927727032
Epoch: 0 global_step 2525 i 2524 Avg loss in epoch(incomplete): 0.235235711732
Epoch: 0 global_step 2550 i 2549 Avg loss in epoch(incomplete): 0.234557141218
Epoch: 0 global_step 2575 i 2574 Avg loss in epoch(incomplete): 0.233887859322
Epoch: 0 global_step 2600 i 2599 Avg loss in epoch(incomplete): 0.23322776219
Epoch: 0 global_step 2625 i 2624 Avg loss in epoch(incomplete): 0.232581459301
Epoch: 0 global_step 2650 i 2649 Avg loss in epoch(incomplete): 0.231946900909
Epoch: 0 global_step 2675 i 2674 Avg loss in epoch(incomplete): 0.231328336986
Epoch: 0 global_step 2700 i 2699 Avg loss in epo

Epoch: 0 global_step 4775 i 4774 Avg loss in epoch(incomplete): 0.202175258167
Epoch: 0 global_step 4800 i 4799 Avg loss in epoch(incomplete): 0.201980660312
Epoch: 0 global_step 4825 i 4824 Avg loss in epoch(incomplete): 0.201788363182
Epoch: 0 global_step 4850 i 4849 Avg loss in epoch(incomplete): 0.201602485023
Epoch: 0 global_step 4875 i 4874 Avg loss in epoch(incomplete): 0.201419401563
Epoch: 0 global_step 4900 i 4899 Avg loss in epoch(incomplete): 0.201235611186
Epoch: 0 global_step 4925 i 4924 Avg loss in epoch(incomplete): 0.201051071465
Epoch: 0 global_step 4950 i 4949 Avg loss in epoch(incomplete): 0.200864682691
Epoch: 0 global_step 4975 i 4974 Avg loss in epoch(incomplete): 0.200684511712
saving
Epoch: 0 global_step 5000 i 4999 Avg loss in epoch(incomplete): 0.20050357081
Epoch: 0 global_step 5000 i 4999 Avg loss in epoch(incomplete): 0.20050357081
Epoch: 0 global_step 5025 i 5024 Avg loss in epoch(incomplete): 0.200329421174
Epoch: 0 global_step 5050 i 5049 Avg loss in ep

Epoch: 0 global_step 7150 i 7149 Avg loss in epoch(incomplete): 0.18979552894
Epoch: 0 global_step 7175 i 7174 Avg loss in epoch(incomplete): 0.189707957762
Epoch: 0 global_step 7200 i 7199 Avg loss in epoch(incomplete): 0.189617131744
Epoch: 0 global_step 7225 i 7224 Avg loss in epoch(incomplete): 0.189534883375
saving
Epoch: 0 global_step 7250 i 7249 Avg loss in epoch(incomplete): 0.189449596424
Epoch: 0 global_step 7250 i 7249 Avg loss in epoch(incomplete): 0.189449596424
Epoch: 0 global_step 7275 i 7274 Avg loss in epoch(incomplete): 0.189367604215
Epoch: 0 global_step 7300 i 7299 Avg loss in epoch(incomplete): 0.189285849482
Epoch: 0 global_step 7325 i 7324 Avg loss in epoch(incomplete): 0.189203282243
Epoch: 0 global_step 7350 i 7349 Avg loss in epoch(incomplete): 0.189119581825
Epoch: 0 global_step 7375 i 7374 Avg loss in epoch(incomplete): 0.189038853421
Epoch: 0 global_step 7400 i 7399 Avg loss in epoch(incomplete): 0.188956545307
Epoch: 0 global_step 7425 i 7424 Avg loss in e

saving
Epoch: 1 global_step 9500 i 804 Avg loss in epoch(incomplete): 0.164872848784
Epoch: 1 global_step 9500 i 804 Avg loss in epoch(incomplete): 0.164872848784
Epoch: 1 global_step 9525 i 829 Avg loss in epoch(incomplete): 0.16487275432
Epoch: 1 global_step 9550 i 854 Avg loss in epoch(incomplete): 0.164829731232
Epoch: 1 global_step 9575 i 879 Avg loss in epoch(incomplete): 0.164835617539
Epoch: 1 global_step 9600 i 904 Avg loss in epoch(incomplete): 0.164843052477
Epoch: 1 global_step 9625 i 929 Avg loss in epoch(incomplete): 0.164866798379
Epoch: 1 global_step 9650 i 954 Avg loss in epoch(incomplete): 0.164888914858
Epoch: 1 global_step 9675 i 979 Avg loss in epoch(incomplete): 0.164903297412
Epoch: 1 global_step 9700 i 1004 Avg loss in epoch(incomplete): 0.16491463985
Epoch: 1 global_step 9725 i 1029 Avg loss in epoch(incomplete): 0.164902459143
saving
Epoch: 1 global_step 9750 i 1054 Avg loss in epoch(incomplete): 0.164901748596
Epoch: 1 global_step 9750 i 1054 Avg loss in epoc

Epoch: 1 global_step 11825 i 3129 Avg loss in epoch(incomplete): 0.164952303636
Epoch: 1 global_step 11850 i 3154 Avg loss in epoch(incomplete): 0.164951063038
Epoch: 1 global_step 11875 i 3179 Avg loss in epoch(incomplete): 0.164948415822
Epoch: 1 global_step 11900 i 3204 Avg loss in epoch(incomplete): 0.164951080967
Epoch: 1 global_step 11925 i 3229 Avg loss in epoch(incomplete): 0.16494443847
Epoch: 1 global_step 11950 i 3254 Avg loss in epoch(incomplete): 0.164935975479
Epoch: 1 global_step 11975 i 3279 Avg loss in epoch(incomplete): 0.164931085606
saving
Epoch: 1 global_step 12000 i 3304 Avg loss in epoch(incomplete): 0.16492766164
Epoch: 1 global_step 12000 i 3304 Avg loss in epoch(incomplete): 0.16492766164
Epoch: 1 global_step 12025 i 3329 Avg loss in epoch(incomplete): 0.164937074908
Epoch: 1 global_step 12050 i 3354 Avg loss in epoch(incomplete): 0.16494261077
Epoch: 1 global_step 12075 i 3379 Avg loss in epoch(incomplete): 0.164948703203
Epoch: 1 global_step 12100 i 3404 Avg

Epoch: 1 global_step 14150 i 5454 Avg loss in epoch(incomplete): 0.164920786528
Epoch: 1 global_step 14175 i 5479 Avg loss in epoch(incomplete): 0.164918804394
Epoch: 1 global_step 14200 i 5504 Avg loss in epoch(incomplete): 0.164915996554
Epoch: 1 global_step 14225 i 5529 Avg loss in epoch(incomplete): 0.164907387004
saving
Epoch: 1 global_step 14250 i 5554 Avg loss in epoch(incomplete): 0.164906795397
Epoch: 1 global_step 14250 i 5554 Avg loss in epoch(incomplete): 0.164906795397
Epoch: 1 global_step 14275 i 5579 Avg loss in epoch(incomplete): 0.164903761775
Epoch: 1 global_step 14300 i 5604 Avg loss in epoch(incomplete): 0.164903590867
Epoch: 1 global_step 14325 i 5629 Avg loss in epoch(incomplete): 0.16490052254
Epoch: 1 global_step 14350 i 5654 Avg loss in epoch(incomplete): 0.164900508671
Epoch: 1 global_step 14375 i 5679 Avg loss in epoch(incomplete): 0.164904170785
Epoch: 1 global_step 14400 i 5704 Avg loss in epoch(incomplete): 0.164905507289
Epoch: 1 global_step 14425 i 5729 

Epoch: 1 global_step 16475 i 7779 Avg loss in epoch(incomplete): 0.164918970372
saving
Epoch: 1 global_step 16500 i 7804 Avg loss in epoch(incomplete): 0.164920085619
Epoch: 1 global_step 16500 i 7804 Avg loss in epoch(incomplete): 0.164920085619
Epoch: 1 global_step 16525 i 7829 Avg loss in epoch(incomplete): 0.16491861543
Epoch: 1 global_step 16550 i 7854 Avg loss in epoch(incomplete): 0.164920925906
Epoch: 1 global_step 16575 i 7879 Avg loss in epoch(incomplete): 0.164920603189
Epoch: 1 global_step 16600 i 7904 Avg loss in epoch(incomplete): 0.164920714564
Epoch: 1 global_step 16625 i 7929 Avg loss in epoch(incomplete): 0.164921935466
Epoch: 1 global_step 16650 i 7954 Avg loss in epoch(incomplete): 0.16492284878
Epoch: 1 global_step 16675 i 7979 Avg loss in epoch(incomplete): 0.164922436187
Epoch: 1 global_step 16700 i 8004 Avg loss in epoch(incomplete): 0.164922870952
Epoch: 1 global_step 16725 i 8029 Avg loss in epoch(incomplete): 0.164918744917
saving
Epoch: 1 global_step 16750 i

KeyboardInterrupt: 

In [64]:
#
# get test data

global_step_to_load_test = 17390


scores = []
labels = []

with tf.Session() as sess:
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=None)
    saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load_test))
    
    idx = 0
    while idx < 45458 / batch_size:
        batch_Xt, batch_Yt = sess.run([seqt_batch, labelt_batch])
        batch_logits = sess.run(logits, feed_dict={X: batch_Xt, Y: batch_Yt})
        scores.append(batch_logits)
        labels.append(batch_Yt)
        #print('asdf')
        idx += 1
        if idx % 100 == 0:
            print(idx)

scores_arr = np.concatenate(scores, axis=0)
labels_arr = np.concatenate(labels, axis=0)

INFO:tensorflow:Restoring parameters from log/motifconv-17390
100
200
300
400
500
600
700
800
900
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: shuffle_batch_1/random_shuffle_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch_1/random_shuffle_queue, Cast_3, Cast_2)]]


In [67]:
accuracies = np.mean((scores_arr > 0).astype(int) == labels_arr.astype(int), axis=0)

print('accuracies:')
print(accuracies)

import sklearn.metrics
print('aucs:')
aucs = [sklearn.metrics.roc_auc_score(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(aucs)

import sklearn.metrics
def prec_recall(ys_true, ys_hat):
    ys, xs, thresholds = sklearn.metrics.precision_recall_curve(ys_true, ys_hat)
    return sklearn.metrics.auc(xs, ys, reorder=True)

print('auprcs:')
auprcs = [prec_recall(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(auprcs)

props = np.mean(labels_arr, axis=0)

import json
ids = sorted(json.loads(info['tf_to_pos'].replace("'", '"')).keys())

from IPython.display import display_pretty, display_html
import pandas as pd
results = pd.DataFrame({'props':props, 'acc':accuracies, 'auprcs':auprcs, 'aucs':aucs}, index=ids).sort_values(by='aucs')
display_html(results.to_html(), raw=True)
results.to_csv('stats.' + modelname + '.tsv', index_label='id', sep='\t')

accuracies:
[ 0.92912088  0.98846154  0.99094505  0.99454945  0.97520879  0.97378022
  0.9752967   0.90749451  0.98142857  0.97835165  0.94134066  0.96804396
  0.94942857  0.96786813  0.90402198  0.94010989  0.94386813  0.98325275
  0.97925275  0.92118681  0.99        0.91628571  0.97127473  0.92942857]
aucs:
[0.45687305433691361, 0.7971208766775193, 0.65130680800264606, 0.50415036904446209, 0.61617300228627991, 0.61123845168467639, 0.51833535945177633, 0.4547736015859114, 0.70696365760110891, 0.65810168362800203, 0.57230948783091395, 0.66478549748009474, 0.48582936860560988, 0.64030893229942132, 0.36862873366530086, 0.37547499128681661, 0.36538954001749241, 0.61694390929798448, 0.66325267355420947, 0.57135079409007383, 0.59488369993864487, 0.41554219182938618, 0.77945814328265295, 0.47860540209671881]
auprcs:
[0.062992902461802072, 0.034374914796597829, 0.015497500929958722, 0.0076064435546530753, 0.035430268805852469, 0.036651741574993672, 0.025535563624608223, 0.083213222270400955, 

,acc,aucs,auprcs,props
SydhHuvecGata2Ucd,0.943868,0.365390,0.040325,0.056132
SydhHuvecCfosUcd,0.904022,0.368629,0.069980,0.095978
SydhHuvecCjun,0.940110,0.375475,0.043733,0.059890
UtaHuvecCtcf,0.916286,0.415542,0.067559,0.083714
HaibGm12891Pu1Pcr1x,0.907495,0.454774,0.083213,0.092505
BroadHuvecCtcf,0.929121,0.456873,0.062993,0.070879
UwHuvecCtcf,0.929429,0.478605,0.066805,0.070571
SydhGm12891NfkbTnfaIggrab,0.949429,0.485829,0.047657,0.050571
HaibGm12891Pax5c20V0416101,0.994549,0.504150,0.007606,0.005451
HaibGm12891Pou2f2Pcr1x,0.975297,0.518335,0.025536,0.024703
